In [5]:
! pip install pymysql

In [1]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re

In [4]:
import pandas
df = pandas.read_csv("../7107.csv")

In [15]:
df

,Country,State,State LGD Code,Year,Month,Settlement of Integrated goods and services tax ( IGST )_avg,Settlement of Integrated goods and services tax ( IGST )_max,Settlement of Integrated goods and services tax ( IGST )_min,Settlement of Integrated goods and services tax ( IGST )_sum,Settlement of Integrated goods and services tax ( IGST )_count,Settlement of Integrated goods and services tax ( IGST )_stddev,Settlement of Integrated goods and services tax ( IGST )_LandAreaWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationWeight,Settlement of Integrated goods and services tax ( IGST )_NumberOfHouseholdsWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationMaleWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationFemaleWeight
0,India,Jammu And Kashmir,1,"Calendar Year (Jan - Dec), 2017","August, 2017",81.591826,81.591826,81.591826,81.591826,1,NaN,81.592,81.592,81.592,81.592,81.592
1,India,Jammu And Kashmir,1,"Calendar Year (Jan - Dec), 2017","September, 2017",139.444544,139.444544,139.444544,139.444544,1,NaN,139.445,139.445,139.445,139.445,139.445
2,India,Jammu And Kashmir,1,"Calendar Year (Jan - Dec), 2017","October, 2017",191.539490,191.539490,191.539490,191.539490,1,NaN,191.539,191.539,191.539,191.539,191.539
3,India,Jammu And Kashmir,1,"Calendar Year (Jan - Dec), 2017","November, 2017",204.687283,204.687283,204.687283,204.687283,1,NaN,204.687,204.687,204.687,204.687,204.687
4,India,Jammu And Kashmir,1,"Calendar Year (Jan - Dec), 2017","December, 2017",182.624872,182.624872,182.624872,182.624872,1,NaN,182.625,182.625,182.625,182.625,182.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,India,The Dadra And Nagar Haveli And Daman And Diu,38,"Calendar Year (Jan - Dec), 2022","February, 2022",36.365185,36.365185,36.365185,36.365185,1,NaN,36.365,36.365,36.365,36.365,36.365
2120,India,The Dadra And Nagar Haveli And Daman And Diu,38,"Calendar Year (Jan - Dec), 2022","March, 2022",79.902823,79.902823,79.902823,79.902823,1,NaN,79.903,79.903,79.903,79.903,79.903
2121,India,The Dadra And Nagar Haveli And Daman And Diu,38,"Calendar Year (Jan - Dec), 2022","April, 2022",55.222938,55.222938,55.222938,55.222938,1,NaN,55.223,55.223,55.223,55.223,55.223
2122,India,The Dadra And Nagar Haveli And Daman And Diu,38,"Calendar Year (Jan - Dec), 2022","May, 2022",45.330816,45.330816,45.330816,45.330816,1,NaN,45.331,45.331,45.331,45.331,45.331


In [7]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="world-athletics-world-athletics.h.aivencloud.com",
  password="AVNS_q4qfMZEYhnIhreAFwYi",
  read_timeout=timeout,
  port=24936,
  user="avnadmin",
  write_timeout=timeout,
)


In [8]:
states = []
cursor = connection.cursor()
cursor.execute("select state from States")
x = cursor.fetchall()
cursor.close()

In [9]:
unique_state_list = [i["state"] for i in x ]

In [106]:
# x

In [16]:
def check_similarity(col_name_sql,col_name_api):
    pass

def match_state_name(input_state, unique_states_list):
    # Initialize variables to store the best match and highest score
    best_match = None
    highest_score = 0

    # Iterate through the unique states list and calculate Jaro-Winkler scores
    for state in unique_states_list:
        similarity_score = jellyfish.jaro_winkler_similarity(input_state, state)
        if similarity_score > highest_score:
            highest_score = similarity_score
            best_match = state
        # print(similarity_score, state)

    # Return the best matching state name
    if(highest_score < 0.7):
        return None
    return best_match

def get_start_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[1] + " " + temp[-1] 
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
        return date_object
    except:
        return None

def get_end_date(string:str):
    try:
        temp = re.split(r"[(),-]",string)
        date_string = "1 " + temp[2] + " " + str(int(temp[-1]) +1)
        date_object = datetime.strptime(date_string, "%d %b %Y").date()
    except:
        return None
    return date_object


In [11]:
def get_month(string: str):
    try:
        temp = re.split(r"[(),-]",string)
        
        date_string = "1 " + temp[-2] + " " + str(int(temp[-1]))
        # print(date_string)
        date_object = datetime.strptime(date_string, "%d %B %Y").date()
    except:
        return None
    return date_object


In [17]:
get_month("December, 2019")

datetime.date(2019, 12, 1)

In [18]:
get_end_date("Financial Year (Apr - Mar), 2017")

datetime.date(2018, 3, 1)

In [27]:
df_per_year = df.drop(["State LGD Code","Month","Country"],axis=1)

In [28]:
df_per_year

,State,Year,Settlement of Integrated goods and services tax ( IGST )_avg,Settlement of Integrated goods and services tax ( IGST )_max,Settlement of Integrated goods and services tax ( IGST )_min,Settlement of Integrated goods and services tax ( IGST )_sum,Settlement of Integrated goods and services tax ( IGST )_count,Settlement of Integrated goods and services tax ( IGST )_stddev,Settlement of Integrated goods and services tax ( IGST )_LandAreaWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationWeight,Settlement of Integrated goods and services tax ( IGST )_NumberOfHouseholdsWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationMaleWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationFemaleWeight
0,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017",81.591826,81.591826,81.591826,81.591826,1,NaN,81.592,81.592,81.592,81.592,81.592
1,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017",139.444544,139.444544,139.444544,139.444544,1,NaN,139.445,139.445,139.445,139.445,139.445
2,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017",191.539490,191.539490,191.539490,191.539490,1,NaN,191.539,191.539,191.539,191.539,191.539
3,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017",204.687283,204.687283,204.687283,204.687283,1,NaN,204.687,204.687,204.687,204.687,204.687
4,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017",182.624872,182.624872,182.624872,182.624872,1,NaN,182.625,182.625,182.625,182.625,182.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022",36.365185,36.365185,36.365185,36.365185,1,NaN,36.365,36.365,36.365,36.365,36.365
2120,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022",79.902823,79.902823,79.902823,79.902823,1,NaN,79.903,79.903,79.903,79.903,79.903
2121,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022",55.222938,55.222938,55.222938,55.222938,1,NaN,55.223,55.223,55.223,55.223,55.223
2122,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022",45.330816,45.330816,45.330816,45.330816,1,NaN,45.331,45.331,45.331,45.331,45.331


In [22]:
# df_per_year
df_per_month = df.drop(["Year","State LGD Code","Country"],axis=1)

In [26]:
# df_per_month
df_per_year

,State,Year,Month,Settlement of Integrated goods and services tax ( IGST )_avg,Settlement of Integrated goods and services tax ( IGST )_max,Settlement of Integrated goods and services tax ( IGST )_min,Settlement of Integrated goods and services tax ( IGST )_sum,Settlement of Integrated goods and services tax ( IGST )_count,Settlement of Integrated goods and services tax ( IGST )_stddev,Settlement of Integrated goods and services tax ( IGST )_LandAreaWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationWeight,Settlement of Integrated goods and services tax ( IGST )_NumberOfHouseholdsWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationMaleWeight,Settlement of Integrated goods and services tax ( IGST )_TotalPopulationFemaleWeight
0,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017","August, 2017",81.591826,81.591826,81.591826,81.591826,1,NaN,81.592,81.592,81.592,81.592,81.592
1,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017","September, 2017",139.444544,139.444544,139.444544,139.444544,1,NaN,139.445,139.445,139.445,139.445,139.445
2,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017","October, 2017",191.539490,191.539490,191.539490,191.539490,1,NaN,191.539,191.539,191.539,191.539,191.539
3,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017","November, 2017",204.687283,204.687283,204.687283,204.687283,1,NaN,204.687,204.687,204.687,204.687,204.687
4,Jammu And Kashmir,"Calendar Year (Jan - Dec), 2017","December, 2017",182.624872,182.624872,182.624872,182.624872,1,NaN,182.625,182.625,182.625,182.625,182.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022","February, 2022",36.365185,36.365185,36.365185,36.365185,1,NaN,36.365,36.365,36.365,36.365,36.365
2120,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022","March, 2022",79.902823,79.902823,79.902823,79.902823,1,NaN,79.903,79.903,79.903,79.903,79.903
2121,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022","April, 2022",55.222938,55.222938,55.222938,55.222938,1,NaN,55.223,55.223,55.223,55.223,55.223
2122,The Dadra And Nagar Haveli And Daman And Diu,"Calendar Year (Jan - Dec), 2022","May, 2022",45.330816,45.330816,45.330816,45.330816,1,NaN,45.331,45.331,45.331,45.331,45.331


In [29]:
# import numpy as np
df_per_year = df_per_year.groupby(["State","Year"]).mean().reset_index()

In [31]:
for i in df_per_year.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_year[i])
        df_per_year = df_per_year.drop([i],axis=1)

In [33]:
for i in df_per_month.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] != "avg"):
        print(df_per_month[i])
        df_per_month = df_per_month.drop([i],axis=1)

In [34]:
df_per_year

,State,Year,Settlement of Integrated goods and services tax ( IGST )_avg
0,Andaman And Nicobar Islands,"Calendar Year (Jan - Dec), 2017",7.008180
1,Andaman And Nicobar Islands,"Calendar Year (Jan - Dec), 2018",12.980731
2,Andaman And Nicobar Islands,"Calendar Year (Jan - Dec), 2019",17.973762
3,Andaman And Nicobar Islands,"Calendar Year (Jan - Dec), 2020",16.757644
4,Andaman And Nicobar Islands,"Calendar Year (Jan - Dec), 2021",18.292241
...,...,...,...
211,West Bengal,"Calendar Year (Jan - Dec), 2018",754.974497
212,West Bengal,"Calendar Year (Jan - Dec), 2019",832.769101
213,West Bengal,"Calendar Year (Jan - Dec), 2020",750.964769
214,West Bengal,"Calendar Year (Jan - Dec), 2021",967.482461


In [23]:
# df_per_year

In [35]:
df_per_year["state"] = df_per_year.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_year["start_date"] = df_per_year.apply(lambda x : get_start_date(x["Year"]),axis=1)
df_per_year["end_date"] = df_per_year.apply(lambda x : get_end_date(x["Year"]),axis=1)


In [37]:
col_name = ["state","start_date","end_date"]
for i in df_per_year.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] == "avg"):
        print(df_per_year[i])
        df_per_year[x[0]] = df_per_year[i]
        col_name.append(x[0])

0         7.008180
1        12.980731
2        17.973762
3        16.757644
4        18.292241
          ...     
211     754.974497
212     832.769101
213     750.964769
214     967.482461
215    1153.706306
Name: Settlement of Integrated goods and services tax ( IGST )_avg, Length: 216, dtype: float64


In [38]:
df_per_year[col_name]

,state,start_date,end_date,Settlement of Integrated goods and services tax ( IGST )
0,andaman and nicobar islands,2017-01-01,2018-12-01,7.008180
1,andaman and nicobar islands,2018-01-01,2019-12-01,12.980731
2,andaman and nicobar islands,2019-01-01,2020-12-01,17.973762
3,andaman and nicobar islands,2020-01-01,2021-12-01,16.757644
4,andaman and nicobar islands,2021-01-01,2022-12-01,18.292241
...,...,...,...,...
211,west bengal,2018-01-01,2019-12-01,754.974497
212,west bengal,2019-01-01,2020-12-01,832.769101
213,west bengal,2020-01-01,2021-12-01,750.964769
214,west bengal,2021-01-01,2022-12-01,967.482461


In [39]:
df_per_year[col_name].to_csv("trans_GST_info_per_year_data_2.csv")

In [ ]:
df_per_month["state"] = df_per_month.apply(lambda x : match_state_name(x["State"],unique_state_list),axis=1)
df_per_month["date"] = df_per_month.apply(lambda x : get_month(x["Month"]),axis=1)
# df_per_month["registered_payer"] = df_per_month.apply(lambda x : x["Gst ( goods and service tax ) payers registered before due date_avg"] +x["Gst ( goods and service tax ) payers registered after due date_avg"] ,axis=1) 
# df_per_month["eligible_payer"] = df_per_month.apply(lambda x : x["Payer eligible for gst ( goods and service tax ) registration_avg"],axis=1)


In [157]:
df_per_year_transformed = df_per_year[["state","start_date","end_date","registered_payer","eligible_payer"]]

In [45]:
col_name = ["state","date"]
for i in df_per_month.keys():
    x = i.split('_')
    if(len(x) == 2 and x[1] == "avg"):
        print(df_per_month[i])
        df_per_month[x[0]] = df_per_month[i]
        col_name.append(x[0])
df_per_month_transformed =  df_per_month[col_name]

0        81.591826
1       139.444544
2       191.539490
3       204.687283
4       182.624872
           ...    
2119     36.365185
2120     79.902823
2121     55.222938
2122     45.330816
2123    -19.675713
Name: Settlement of Integrated goods and services tax ( IGST )_avg, Length: 2124, dtype: float64


In [46]:
df_per_month_transformed.dropna()

,state,date,Settlement of Integrated goods and services tax ( IGST )
0,jammu and kashmir,2017-08-01,81.591826
1,jammu and kashmir,2017-09-01,139.444544
2,jammu and kashmir,2017-10-01,191.539490
3,jammu and kashmir,2017-11-01,204.687283
4,jammu and kashmir,2017-12-01,182.624872
...,...,...,...
2119,the dadra and nagar haveli and daman and diu,2022-02-01,36.365185
2120,the dadra and nagar haveli and daman and diu,2022-03-01,79.902823
2121,the dadra and nagar haveli and daman and diu,2022-04-01,55.222938
2122,the dadra and nagar haveli and daman and diu,2022-05-01,45.330816


In [164]:
df_per_year_transformed.dropna()

,state,start_date,end_date,registered_payer,eligible_payer
0,andaman and nicobar islands,2017-04-01,2018-03-01,2596.500000,3388.833333
1,andaman and nicobar islands,2018-04-01,2019-03-01,2871.958333,4107.000000
2,andaman and nicobar islands,2019-04-01,2020-03-01,2914.541667,3570.041667
3,andaman and nicobar islands,2020-04-01,2021-03-01,3049.166667,3627.708333
4,andaman and nicobar islands,2021-04-01,2022-03-01,3207.208333,3562.750000
...,...,...,...,...,...
211,west bengal,2018-04-01,2019-03-01,375588.041667,437264.333333
212,west bengal,2019-04-01,2020-03-01,411276.708333,492672.916667
213,west bengal,2020-04-01,2021-03-01,408896.416667,511896.625000
214,west bengal,2021-04-01,2022-03-01,404378.250000,455467.500000


In [48]:
# df_per_year_transformed.to_csv("trans_register_payers_per_year_data.csv")
df_per_month_transformed.to_csv("trans_register_payers_per_month_data_2.csv")